# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [8]:
np.mean(pd_data['class_buys_computer'] == 'yes')

0.6428571428571429

In [11]:
def get_gini(df, label):
    
    p_y = np.mean(df[label] == 'yes') # 이 데이터의 경우 yes , no 이진분류 문제기 때문에 y==true일 확률
    p_n = 1 - p_y # y==no일 확률을 구해준 후 
    gini = 1 - (p_y**2 + p_n**2) # gini 계수 구하는 공식에 의하여 값을 구한다.
    
    return gini    

In [12]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [13]:
def get_binary_split(df, attribute):
    result = []
    values = df[attribute].unique()  # feature의 unique값들을 이진 분류로 만들기 위해 완전 탐색을 이용하여 
    
    for i in range(len(values)) :     
        result.append([values[i]])     # 모든 조합을 구한 후 return해 준다.
        for j in range(i+1,len(values) ):
            result.append([values[i],values[j]])
    
    return result

In [14]:
get_binary_split(pd_data,"age")

[['youth'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged'],
 ['middle_aged', 'senior'],
 ['senior']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [45]:
def get_attribute_gini_index(df, attribute, label):
    
    result = {}
    categories = get_binary_split(df, attribute) # 앞에서 정의한 함수로 이진분류의 경우들을 categories에 담고 
    
    for category in categories :
        y = df[df[attribute].isin(category)] # categories에 list로 저장되어 있으므로 isin함수를 이용하여 df의 변수에서 category에 해당하는 경우와 
        n = df[~df[attribute].isin(category)]# 아닌경우로 나눈다.
        
        gini = len(y)/len(df) * get_gini(y, label) + len(n)/len(df) * get_gini(n, label)
        # 그 후 y = true, false에 대하여 기존에 정의한 get_gini로 gini index를 구한후 
        # 각각의 category에 대하여 gini index를 비교한다.

        result.update({'&'.join(category) : gini})
    
    return result

In [46]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")# 결과는 다음과 같다.

{'youth': 0.3936507936507937,
 'youth&middle_aged': 0.45714285714285713,
 'youth&senior': 0.35714285714285715,
 'middle_aged': 0.35714285714285715,
 'middle_aged&senior': 0.3936507936507937,
 'senior': 0.45714285714285713}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [47]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())  # gini index에 의하여 class split 기준은 middle_aged / youth&senior 이 적당하다.

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [48]:
##문제1 답안

get_binary_split(pd_data, 'income') # income의 unique 값들에 대하여 이진 분류

[['high'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium'],
 ['medium', 'low'],
 ['low']]

In [64]:
##문제2 답안
features = list(pd_data.columns[:-1]) # target 변수인 'class_buys_computer'을 제외한 변수들을 저장하여 각각 gini index를 구해준다.

categories = []
min_ginis = []

for f in features : 
    cate, gini = min(get_attribute_gini_index(pd_data, f, 'class_buys_computer').items()) 
    # get_attribute_gini_index를 이용하여 각 분류 값에 대한 gini index를 구하여 저장을 한 후 np.argmin을 이용하여 최소 gini index를
    # 가지는 index 값을 불러와 print를 해준다. 
    categories.append(cate)
    min_ginis.append(gini)

idx = np.argmin(min_ginis)

print(f"중요한 변수 : {features[idx]}, 해당 변수 & gini 계수 : {categories[idx], min_ginis[idx]}")
# 그 결과로 age의 변수가 중요하며 middle_aged | youth & senior 로 분류가 됨을 확인할 수 있다.

중요한 변수 : age, 해당 변수 & gini 계수 : ('middle_aged', 0.35714285714285715)


In [65]:
##문제3 답안
features.pop(idx)

new_features = features # 위에서 분류한 age를 제거후 새로운 변수들을 list로 정의하여 다음 분류 기준을 정한다.
df_y = pd_data[pd_data['age'] == 'middle_aged'] # 위에서 나온 age에 middle_aged 기준으로 데이터를 분류한다. 
df_n = pd_data[pd_data['age'] != 'middle_aged'] # youth & senior
 
 

In [82]:
df_y, df_n

(            age  income student credit_rating class_buys_computer
 2   middle_aged    high      no          fair                 yes
 6   middle_aged     low     yes     excellent                 yes
 11  middle_aged  medium      no     excellent                 yes
 12  middle_aged    high     yes          fair                 yes,
        age  income student credit_rating class_buys_computer
 0    youth    high      no          fair                  no
 1    youth    high      no     excellent                  no
 3   senior  medium      no          fair                 yes
 4   senior     low     yes          fair                 yes
 5   senior     low     yes     excellent                  no
 7    youth  medium      no          fair                  no
 8    youth     low     yes          fair                 yes
 9   senior  medium     yes          fair                 yes
 10   youth  medium     yes     excellent                 yes
 13  senior  medium      no     excellent   

In [83]:
categories = []
min_ginis = []

for f in new_features : # 위와 같은 방식으로 변수들에 대하여 최소 gini index를 가지는 분류 기준과 gini index를 저장하여 
    cate, gini = min(get_attribute_gini_index(df_y, f, 'class_buys_computer').items())
    categories.append(cate)
    min_ginis.append(gini)
     
    # index를 구한 후 결과를 확인해 본다. 
idx = np.argmin(min_ginis)

print(f"중요한 변수 : {features[idx]}, 해당 변수 & gini 계수 : {categories[idx], min_ginis[idx]}")
# 그 결과 age == middle_aged의 경우 income 변수 중 high | low, medium이 분류된다.
result_y = [features[idx], categories[idx], min_ginis[idx]]

중요한 변수 : income, 해당 변수 & gini 계수 : ('high', 0.0)


In [78]:
categories = []
min_ginis = []

for f in new_features :
    cate, gini = min(get_attribute_gini_index(df_n, f, 'class_buys_computer').items()) 
    categories.append(cate)
    min_ginis.append(gini)

idx = np.argmin(min_ginis)

# age != middle_aged의 경우  income 변수가 가장 낮은 gini index를 가지고 그 중 no를 기준으로 분류가 진행된다. (no | yes)
print(f"중요한 변수 : {features[idx]}, 해당 변수 & gini 계수 : {categories[idx], min_ginis[idx]}")
result_n = [features[idx], categories[idx], min_ginis[idx]]

중요한 변수 : student, 해당 변수 & gini 계수 : ('no', 0.31999999999999984)


In [81]:
result = pd.DataFrame(data = [result_y,result_n], columns = ['feature', 'category', 'gini_index'])
result

,feature,category,gini_index
0,income,high,0.00
1,student,no,0.32
